# Natural Language Processing in Action

## 2. Word Tokenization

In [44]:
# Example sentence
sentence = "Thomas Jefferson began building Monticello at the age of 26."

In [45]:
sentence.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [46]:
str.split(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

### One-hot vector を作ってみる

In [47]:
import numpy as np

In [48]:
!poetry add pandas 

The following packages are already present in the pyproject.toml and will be skipped:

  • pandas

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [49]:
import pandas as pd

In [50]:
token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))
num_tokens = len(token_sequence)
vocab_size = len(vocab)

In [51]:
onehot_vectors = np.zeros((num_tokens, vocab_size), int)

In [53]:
for i, word in enumerate(token_sequence):
    onehot_vectors[i, vocab.index(word)] = 1

onehot_vectors

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [54]:
df = pd.DataFrame(onehot_vectors, columns=vocab)

In [55]:
df

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


### Bag of Words を作ってみる

In [56]:
sentences = (
"Thomas Jefferson began building Monticello at the age of 26.\n"
"Construction was done mostly by local masons and carpenters.\n"
"He moved into the South Pavilion in 1770.\n"
"Turning Monticello into a neoclassical masterpiece was Jefferson's obsession."
)

In [57]:
corpus = {}

In [58]:
for i, sent in enumerate(sentences.split('\n')):
    corpus[f"sent{i}"] = dict((tok, 1) for tok in sent.split())

In [59]:
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T

In [60]:
df[df.columns[:10]]

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent0,1,1,1,1,1,1,1,1,1,1
sent1,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0


In [61]:
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.,...,South,Pavilion,in,1770.,Turning,a,neoclassical,masterpiece,Jefferson's,obsession.
sent0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0,...,1,1,1,1,0,0,0,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1


In [62]:
df = df.T

In [63]:
df.sent0.dot(df.sent1)

0

In [64]:
df.sent0.dot(df.sent2)

1

In [65]:
df.sent0.dot(df.sent3)

1

### 正規表現でトークナイズを改善する

In [66]:
import re
pattern = re.compile(r"[-\s.,;!?]+")
tokens = pattern.split(sentence)
tokens = [x for x in tokens if x and x not in "- \t\n.,;!?"]
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

### ライブラリを使ってトークナイズする

各ライブラリの特徴は次の通り。
- spaCy: Accurate , flexible, fast, Python
- Stanford CoreNLP: More accurate, less flexible, fast, depends on Java 8
- NLTK: Standard used by many NLP contests and comparisons, popular, Python

#### spaCyを使ってみる

In [67]:
import spacy
# 英語用のトークナイザ等を読み込む
nlp = spacy.load("en_core_web_sm")
doc = nlp(sentence)
[token for token in doc]

[Thomas, Jefferson, began, building, Monticello, at, the, age, of, 26, .]

#### NLTKを使ってみる

In [68]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r"\w+|$[0-9.]+|\S+")
tokenizer.tokenize(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '.']

## NLTK の Treebank Tokenizer を使ってみる
wasn't のような短縮形を was n't のように分割してくれる。

In [69]:
from nltk.tokenize import TreebankWordTokenizer
sentence2 = """Monticello wasn't designed as UNESCO World Heritage Site untill 1987."""
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentence2)

['Monticello',
 'was',
 "n't",
 'designed',
 'as',
 'UNESCO',
 'World',
 'Heritage',
 'Site',
 'untill',
 '1987',
 '.']

## n-gramで語彙を広げる

"ice cream" などのように複数単語で成り立っている言葉は、単語でトークナイズしてしまうとそのままでは文書中の意図した意味にならない。こういった言葉をn-gramを使って扱えるようにする。

In [72]:
from nltk.util import ngrams

In [74]:
list(ngrams(tokens, 2))

[('Thomas', 'Jefferson'),
 ('Jefferson', 'began'),
 ('began', 'building'),
 ('building', 'Monticello'),
 ('Monticello', 'at'),
 ('at', 'the'),
 ('the', 'age'),
 ('age', 'of'),
 ('of', '26')]

In [75]:
list(ngrams(tokens, 3))

[('Thomas', 'Jefferson', 'began'),
 ('Jefferson', 'began', 'building'),
 ('began', 'building', 'Monticello'),
 ('building', 'Monticello', 'at'),
 ('Monticello', 'at', 'the'),
 ('at', 'the', 'age'),
 ('the', 'age', 'of'),
 ('age', 'of', '26')]

### Stop words

文章の本質的な意味に寄与せず、かつ多くの文書で出てくる言葉をStop Wordsという。
例えば、以下のようなものがある。
- a, an
- the, this
- and, or
- of, on
歴史的に計算負荷をへらすことを目的などにしてStop wordsは除去されることが多いが、わずかながらにも文書の意味に寄与することがある。例えば、以下のような文では、Stop wordsの有無で文意が変わってしまう。
- Mark reported to the CEO.
- Suzanne reported as the CEO to the board.


In [76]:
# NLTKのStop Wordsには次のようなものが含まれている。
import nltk
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")
len(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akitanak/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [77]:
stop_words[:7]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours']

## 語彙を正規化する

文章中には、単語の活用等によって異なるスペリングでも同じ意味を持つことがある。このようなものを一つの単語に統一するためのテクニックとして、以下のようなものがある。
- Case Folding
- Stemming
- Lemmatization

### Case Folding
Capitalizatinを修正して単語のスペルを揃える。すべてをlower caseにしてしまうと、upper caseであることに意味があるものが失われてしまうので注意。文頭の単語のみ小文字にする等の工夫が必要。

In [79]:
print(tokens)
normalized_tokens = [x.lower() for x in tokens]
print(normalized_tokens)

['Thomas', 'Jefferson', 'began', 'building', 'Monticello', 'at', 'the', 'age', 'of', '26']
['thomas', 'jefferson', 'began', 'building', 'monticello', 'at', 'the', 'age', 'of', '26']


### Stemming

複数形や所有格等の活用形における共通的な単語の stem を特定する手法である。たとえば、 "house" と "houses"、"house's" を "house" に揃える。スペリングを元に機械的に stem に変換するため、文中の意味を失うこともある。
検索エンジンなどで利用すると、対象となる文書が増えるためrecallは改善するが、その分余計な文書も含まれるようになりprecisionに悪影響を与えることになる。

In [81]:
def stem(words):
    return " ".join([re.findall('^(.*ss|.*?)(s)?$', word)[0][0].strip("'") for word in words.lower().split()])

print(stem("houses"))
print(stem("Doctor House's calls"))

house
doctor house call


ポピュラーなstemmingアルゴリズムとして、Porter stemmer と Snowball stemmer がある。どちらもMartine Porterによって考案されたアルゴリズムで Snowball stemmer は Porter stemmer の拡張版である。
[ここ](https://github.com/jedijulia/porter-stemmer/blob/ master/stemmer.py)にPorter stemmerのpython実装がある。
nltk が Porter stemmer の実装を提供している。

In [87]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
" ".join([stemmer.stem(w).strip("'") for w in "dish washer's washed dished".split()])

'dish washer wash dish'

### Lemmatization

意味的に同じ幹に正規化する手法である。Lemmatizer は Part of Speech(POS)(品詞)も単語と一緒に受け取り処理を行う。Lemmatization は Stemming よりも精度が高くなることが期待できる。

In [89]:
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("better"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("good", pos="a"))
print(lemmatizer.lemmatize("goods", pos="a"))
print(lemmatizer.lemmatize("goods", pos="n"))
print(lemmatizer.lemmatize("goodness", pos="n"))
print(lemmatizer.lemmatize("goodness", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))

better
good
good
goods
good
goodness
goodness
best


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/akitanak/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
